> **Trabalho Prático - Módulo 2**

---



Instalando o Pypspark

In [1]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 46 kB/s 
     |████████████████████████████████| 199 kB 63.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=479690bd36f2ac3d5dd70e471bffd9f8dab758c4e59ef22b92ae60dbc5c3b8ec
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

In [23]:
from pyspark.sql import functions as f

In [83]:
from pyspark.sql.functions import *

In [4]:
spark = SparkSession.builder.getOrCreate()

Importando a base de dados

In [5]:
df = (spark.read
      .format("csv")
      .option("header", "True")
      .option("inferSchema", "True")
      .load("/content/all_stocks_5yr.csv")
    
)

In [46]:
df.show(5)

+-------------------+-----+-----+-----+-----+--------+----+
|               date| open| high|  low|close|  volume|Name|
+-------------------+-----+-----+-----+-----+--------+----+
|2013-02-08 00:00:00|15.07|15.12|14.63|14.75| 8407500| AAL|
|2013-02-11 00:00:00|14.89|15.01|14.26|14.46| 8882000| AAL|
|2013-02-12 00:00:00|14.45|14.51| 14.1|14.27| 8126000| AAL|
|2013-02-13 00:00:00| 14.3|14.94|14.25|14.66|10259500| AAL|
|2013-02-14 00:00:00|14.94|14.96|13.16|13.99|31879900| AAL|
+-------------------+-----+-----+-----+-----+--------+----+
only showing top 5 rows



##Pergunta 1
Quantos registros há na planilha?

In [43]:
df.count()

619040

##Pergunta 2
Quantos registros há na planilha para a ação da Apple (AAPL)?

In [32]:
df.filter(df.Name == 'AAPL').count()

1259

##Pergunta 3
Quantas empresas distintas têm registros nessa planilha?

In [33]:
from pyspark.sql.functions import countDistinct

In [42]:
df.select(countDistinct("Name")).show()

+--------------------+
|count(DISTINCT Name)|
+--------------------+
|                 505|
+--------------------+



##Pergunta 4
Com qual frequência o preço de uma ação no fechamento é maior do que o preço na abertura?

In [13]:
newdf = df.filter(df.close > df.open) #separando os dados em que o fechamento é maior que a abertura

In [20]:
resultado = (newdf.count()*100)/df.count()#calculo de porcentagem
resultado

51.52655724993539

##Pergunta 5
Qual o maior valor das ações da Apple (AAPL) na história?

In [69]:
df.filter(df.Name=='AAPL').describe().show()

+-------+------------------+------------------+------------------+------------------+-------------------+----+
|summary|              open|              high|               low|             close|             volume|Name|
+-------+------------------+------------------+------------------+------------------+-------------------+----+
|  count|              1259|              1259|              1259|              1259|               1259|1259|
|   mean|109.05542891183475|109.95111834789516|108.14158888006372|109.06669849086583|5.404789973550437E7|null|
| stddev| 30.54922002458826|30.686186407983914|30.376223585852472|30.556811676964696|  3.3468353335784E7|null|
|    min|           55.4242|           57.0857|           55.0142|           55.7899|           11475922|AAPL|
|    max|            179.37|             180.1|            178.25|            179.26|          266833581|AAPL|
+-------+------------------+------------------+------------------+------------------+-------------------+----+



##Pergunta 6
Qual ação tem a maior volatilidade? Uma forma é medir o desvio-padrão do preço de fechamento de cada ação e considerar a ação de maior desvio-padrão.

In [83]:
dp = df.groupBy("Name").agg(f.stddev_pop("close"))
dp.sort("stddev_pop(close)", ascending = False).show()

+-----+------------------+
| Name| stddev_pop(close)|
+-----+------------------+
| PCLN|  320.406150879457|
| AMZN| 282.3881803736693|
|GOOGL|187.49938385112065|
| GOOG|166.33516327237143|
|  MTD|131.34502636056496|
|  AZO|130.69766049524412|
|  CMG|130.22887354752876|
| EQIX| 95.17498456884599|
| REGN| 94.03197894020944|
| CHTR| 83.60377683704122|
| ISRG| 72.94701133296836|
| AVGO|   71.387854485383|
| ULTA|  67.1251720790448|
|  NOC| 66.36955373928866|
|  SHW| 64.49495587401339|
|  LMT|62.387238088468635|
|  BLK| 62.31212284476813|
| NVDA|  59.6009577776698|
| ORLY|59.071606636060984|
|  AGN| 59.00290341436256|
+-----+------------------+
only showing top 20 rows



##Pergunta 7
Qual o dia com maior volume total de negociação da bolsa?

In [28]:
df.groupBy("date").sum("volume").sort("sum(volume)", ascending = False).show()


+-------------------+-----------+
|               date|sum(volume)|
+-------------------+-----------+
|2015-08-24 00:00:00| 4607945196|
|2016-06-24 00:00:00| 4367393052|
|2015-12-18 00:00:00| 4124454411|
|2016-01-20 00:00:00| 4087629753|
|2018-02-06 00:00:00| 4072080890|
|2016-11-10 00:00:00| 4060601612|
|2014-10-15 00:00:00| 3993171524|
|2013-06-21 00:00:00| 3983923288|
|2015-09-18 00:00:00| 3962050449|
|2016-11-09 00:00:00| 3915089371|
|2016-01-15 00:00:00| 3787884056|
|2017-12-15 00:00:00| 3786992731|
|2016-02-08 00:00:00| 3759709109|
|2016-03-18 00:00:00| 3755746256|
|2016-01-29 00:00:00| 3740170664|
|2014-03-21 00:00:00| 3611991561|
|2018-02-05 00:00:00| 3598437288|
|2013-03-15 00:00:00| 3569550899|
|2016-12-16 00:00:00| 3564871264|
|2016-02-11 00:00:00| 3547898496|
+-------------------+-----------+
only showing top 20 rows



##Pergunta 8
Qual a ação mais negociada da bolsa, em volume de transações?

In [37]:
df.groupby("Name").sum("volume").sort("sum(volume)", ascending = False).show()

+-----+------------+
| Name| sum(volume)|
+-----+------------+
|  BAC|117884953591|
| AAPL| 68046305767|
|   GE| 53023644686|
|    F| 43388129992|
|   FB| 43258314899|
| MSFT| 42641653597|
|  AMD| 40941468586|
|   MU| 38082749687|
| INTC| 36922332801|
| CSCO| 36075826186|
|  PFE| 35527608853|
|    T| 31850598073|
|  CHK| 31421758366|
|CMCSA| 30630585625|
|  FCX| 29648463667|
|    C| 26044628798|
|  WFC| 23305954532|
|  JPM| 20885592857|
|   RF| 20645650500|
| NFLX| 20405882026|
+-----+------------+
only showing top 20 rows



##Pergunta 9
Quantas ações começam com a letra “A”?

In [68]:
df.select("Name").distinct().filter(df.Name.startswith("A")).count()

59

##Pergunta 10
Com qual frequência o preço mais alto do dia da ação também é o preço de fechamento?

In [74]:
preco = df.filter(df.close == df.high)#filtra preços iguais
preco.count()

7420

In [73]:
precofinal = (preco.count()*100)/df.count()
precofinal

1.1986301369863013

##Pergunta 11
Em qual dia a ação da Apple mais subiu entre a abertura e o fechamento, de forma absoluta?

In [88]:
aapl = df.filter(df.Name == "AAPL").withColumn("diferenca", df.close - df.open)

In [91]:
aapl.orderBy(aapl.diferenca.desc()).show()

+-------------------+-------+-------+-------+-------+---------+----+------------------+
|               date|   open|   high|    low|  close|   volume|Name|         diferenca|
+-------------------+-------+-------+-------+-------+---------+----+------------------+
|2015-08-24 00:00:00|  94.87|  108.8|   92.0| 103.12|162206292|AAPL|              8.25|
|2018-02-06 00:00:00| 154.83| 163.72|  154.0| 163.03| 68243838|AAPL| 8.199999999999989|
|2017-05-08 00:00:00| 149.03|  153.7| 149.03| 153.01| 48752413|AAPL|3.9799999999999898|
|2014-12-09 00:00:00| 110.19|  114.3| 109.35| 114.12| 60208036|AAPL| 3.930000000000007|
|2017-10-27 00:00:00| 159.29|  163.6|  158.7| 163.05| 44454160|AAPL|3.7600000000000193|
|2015-12-04 00:00:00| 115.29| 119.25| 115.11| 119.03| 57776977|AAPL| 3.739999999999995|
|2017-12-11 00:00:00|  169.2| 172.89| 168.79| 172.67| 35273759|AAPL| 3.469999999999999|
|2016-11-16 00:00:00|  106.7| 110.23|  106.6| 109.99| 58840522|AAPL| 3.289999999999992|
|2015-07-22 00:00:00| 121.99|  1

##Pergunta 12
Em média, qual o volume diário de transações das ações da AAPL?

In [13]:
aapl = df.filter(df.Name == "AAPL")

In [14]:
aapl.agg({"volume":"avg"}).show()

+-------------------+
|        avg(volume)|
+-------------------+
|5.404789973550437E7|
+-------------------+



##Pergunta 13
Quantas ações tem 1, 2, 3, 4 e 5 caracteres em seu nome, respectivamente?

In [24]:
from pyspark.sql.functions import col,length,trim

In [45]:
unic = df.select("Name").distinct()
unic.show()

+----+
|Name|
+----+
|ALXN|
| GIS|
|   K|
| LEN|
| AIV|
| AVY|
|BF.B|
| AXP|
|  CI|
| IRM|
|INFO|
| BLK|
|  EA|
|ESRX|
| DUK|
| HAS|
|FLIR|
| EMN|
|EVHC|
| KIM|
+----+
only showing top 20 rows



In [57]:
unic2 = unic.select(length(col("Name")))
unic2.show()

+------------+
|length(Name)|
+------------+
|           4|
|           3|
|           1|
|           3|
|           3|
|           3|
|           4|
|           3|
|           2|
|           3|
|           4|
|           3|
|           2|
|           4|
|           3|
|           3|
|           4|
|           3|
|           4|
|           3|
+------------+
only showing top 20 rows



In [59]:
unic2.groupby("length(Name)").count().show()

+------------+-----+
|length(Name)|count|
+------------+-----+
|           1|   10|
|           3|  323|
|           5|    5|
|           4|  117|
|           2|   50|
+------------+-----+



##Pergunta 14
Qual a ação menos negociada da bolsa, em volume de transações?

In [15]:
df.groupby("Name").sum("volume").sort("sum(volume)", ascending=True).show()

+----+-----------+
|Name|sum(volume)|
+----+-----------+
|APTV|   92947779|
| BHF|  144878392|
| MTD|  214489754|
| AZO|  397585127|
|WLTW|  414734590|
| HII|  418791148|
| DXC|  424529395|
|  RE|  434037215|
| ESS|  477004832|
| SNA|  530283843|
| FRT|  531047903|
|ANSS|  552950012|
| IFF|  558773791|
| AMG|  568888091|
| COO|  569985137|
|  IT|  578515652|
| ROP|  579470940|
| AYI|  608654295|
| FTV|  610590073|
|BHGE|  628313042|
+----+-----------+
only showing top 20 rows



##Pergunta 15
Qual dessas afirmações NÃO ilustra uma diferença entre as abstrações RDD e Dataframe?